In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wiki-med-esp/kaggle_outputs/tsv_output/train_medicine_first_strategy.tsv
/kaggle/input/wiki-med-esp/kaggle_outputs/tsv_output/train_wikicorpus_second_strategy.tsv
/kaggle/input/wiki-med-esp/kaggle_outputs/tsv_output/test_medicine.tsv
/kaggle/input/wiki-med-esp/kaggle_outputs/tsv_output/train_medicine_second_strategy.tsv
/kaggle/input/wiki-med-esp/kaggle_outputs/tsv_output/train_wikicorpus_first_strategy.tsv
/kaggle/input/wiki-med-esp/kaggle_outputs/tsv_output/test_wikicorpus.tsv


In [2]:
def head_of_tsv(file_path, num_lines=5):
    with open(file_path, 'r', encoding='utf-8') as file:
        for _ in range(num_lines):
            print(file.readline().strip().split('\t'))

head_of_tsv('/kaggle/input/wiki-med-esp/kaggle_outputs/tsv_output/test_wikicorpus.tsv', 5)


['source', 'target']
['La iglesia contiene numerosas obras de harte', 'La iglesia contiene numerosas obras de arte']
['Otras obras notables shon las pinturas del Cavalier d Arpino La Virgen María con los santos Francisco de Asís y Agustín y el altar con la Misa de San Gregorio , de Jacopo Zucchi', 'Otras obras notables son las pinturas del Cavalier d Arpino La Virgen María con los santos Francisco de Asís y Agustín y el altar con la Misa de San Gregorio , de Jacopo Zucchi']
['En esta parroquia tamvién desarrolla alguna de sus actividades la Comunidad de Sant Egidio', 'En esta parroquia también desarrolla alguna de sus actividades la Comunidad de Sant Egidio']
['Página oficia ; ENDOFARTICLE', 'Página oficial ; ENDOFARTICLE']


In [3]:
import os
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    TrainingArguments, 
    Trainer,
    DataCollatorForSeq2Seq
)
from tqdm import tqdm
import logging

2025-07-23 13:27:35.498890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753277255.837957      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753277255.939593      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
# Configuration
MODEL_NAME = "jorgeortizfuentes/spanish-spellchecker-t5-base-wiki200000"
DATA_DIR = "/kaggle/input/wiki-med-esp/kaggle_outputs/tsv_output"
OUTPUT_DIR = "./fine_tuned_t5_spellchecker"
MAX_LENGTH = 512

In [6]:
# 🔥 TESTING MODE: Set fraction of data to use (0.1 = 10%, 1.0 = 100%)
DATA_FRACTION = 1.0  # Change this to control data size
USE_FRACTION = True  # Set to False to use full dataset

In [7]:
# Training files to use (first strategy only)
TRAIN_FILES = [
    "train_medicine_first_strategy.tsv",
    "train_wikicorpus_first_strategy.tsv"
]

In [8]:
print("🚀 Starting T5 Spanish Spellchecker Fine-tuning")
if USE_FRACTION:
    print(f"🧪 TESTING MODE: Using {DATA_FRACTION*100}% of data")
else:
    print("📊 FULL MODE: Using complete dataset")
print("=" * 60)

# 1. Load tokenizer and model
print("📥 Loading tokenizer and model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

🚀 Starting T5 Spanish Spellchecker Fine-tuning
🧪 TESTING MODE: Using 100.0% of data
📥 Loading tokenizer and model...


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [9]:
# Ensure model is in training mode
model.train()
print(f"✅ Model loaded: {MODEL_NAME}")

✅ Model loaded: jorgeortizfuentes/spanish-spellchecker-t5-base-wiki200000


In [10]:
# 2. Load and prepare dataset
print("\n📊 Loading dataset files...")
all_data = []


📊 Loading dataset files...


In [11]:
for file_name in TRAIN_FILES:
    file_path = os.path.join(DATA_DIR, file_name)
    if os.path.exists(file_path):
        print(f"Loading {file_name}...")
        df = pd.read_csv(file_path, delimiter='\t')
        
        # Drop rows with missing values
        df = df.dropna()
        
        print(f"  - {file_name}: {len(df)} samples")
        all_data.append(df)
    else:
        print(f"❌ Warning: {file_path} not found!")

# Concatenate all data
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"\n📈 Total combined samples: {len(combined_df)}")
    
    # 🔥 Apply data fraction if enabled
    if USE_FRACTION and DATA_FRACTION < 1.0:
        original_size = len(combined_df)
        # Shuffle and take fraction
        combined_df = combined_df.sample(frac=DATA_FRACTION, random_state=42).reset_index(drop=True)
        print(f"🧪 TESTING MODE: Using {DATA_FRACTION*100}% of data")
        print(f"   Reduced from {original_size} to {len(combined_df)} samples")
    else:
        print(f"📊 Using full dataset: {len(combined_df)} samples")
        
else:
    raise FileNotFoundError("No training files found!")

# Convert to dataset
dataset = Dataset.from_pandas(combined_df)
print(f"✅ Dataset created with {len(dataset)} samples")

Loading train_medicine_first_strategy.tsv...
  - train_medicine_first_strategy.tsv: 156089 samples
Loading train_wikicorpus_first_strategy.tsv...
  - train_wikicorpus_first_strategy.tsv: 9755938 samples

📈 Total combined samples: 9912027
📊 Using full dataset: 9912027 samples
✅ Dataset created with 9912027 samples


In [12]:
# 3. Tokenization function
def tokenize_function(examples):
    # Tokenize inputs (source - noisy text)
    model_inputs = tokenizer(
        examples["source"], 
        max_length=MAX_LENGTH, 
        truncation=True, 
        padding=False  # Don't pad during tokenization
    )
    
    # Tokenize targets (target - clean text) - using text_target parameter
    labels = tokenizer(
        text_target=examples["target"], 
        max_length=MAX_LENGTH, 
        truncation=True, 
        padding=False  # Don't pad during tokenization
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
# 4. Apply tokenization
print("\n🔄 Tokenizing dataset...")
tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True,
    batch_size=4000,  # Process in larger batches
    remove_columns=dataset.column_names,
    num_proc=1  # Single process to avoid issues
)

print(f"✅ Tokenization completed!")
print(f"Dataset size: {len(tokenized_dataset)}")


🔄 Tokenizing dataset...


Map:   0%|          | 0/9912027 [00:00<?, ? examples/s]

✅ Tokenization completed!
Dataset size: 9912027


In [14]:
# # Log token statistics
# sample_size = min(100, len(tokenized_dataset))
# sample_input_lengths = [len(x) for x in tokenized_dataset["input_ids"][:sample_size]]
# sample_label_lengths = [len(x) for x in tokenized_dataset["labels"][:sample_size]]

# print(f"📊 Token Statistics (first {sample_size} samples):")
# print(f"  - Input length: min={min(sample_input_lengths)}, max={max(sample_input_lengths)}, avg={sum(sample_input_lengths)/len(sample_input_lengths):.1f}")
# print(f"  - Label length: min={min(sample_label_lengths)}, max={max(sample_label_lengths)}, avg={sum(sample_label_lengths)/len(sample_label_lengths):.1f}")

In [19]:
# 5. Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

# 6. Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="no",
    logging_steps=20 if USE_FRACTION else 50,
    save_steps=100 if USE_FRACTION else 500,
    save_total_limit=2,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=2 if USE_FRACTION else 3,
    warmup_steps=50 if USE_FRACTION else 100,
    weight_decay=0.01,
    logging_dir=f"{OUTPUT_DIR}/logs",
    save_strategy="steps",
    report_to=[],  # Empty list instead of None
    dataloader_num_workers=0,
    remove_unused_columns=False,
    push_to_hub=False,
    fp16=False,
    dataloader_pin_memory=False,
    run_name="t5_spellchecker_training",  # Explicit run name
    disable_tqdm=False,  # Keep progress bars
)

# 7. Initialize trainer
print("\n🏋️ Initializing trainer...")


🏋️ Initializing trainer...


In [20]:
# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    processing_class=tokenizer,  # Use processing_class instead of tokenizer
    data_collator=data_collator,
)

print("✅ Trainer initialized successfully!")

✅ Trainer initialized successfully!


In [21]:
# 8. Start training
print("\n🚀 Starting training...")
print(f"Total training samples: {len(tokenized_dataset)}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Gradient accumulation steps: {training_args.gradient_accumulation_steps}")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Epochs: {training_args.num_train_epochs}")

steps_per_epoch = len(tokenized_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)
total_steps = steps_per_epoch * training_args.num_train_epochs
print(f"Steps per epoch: {steps_per_epoch}")
print(f"Total steps: {total_steps}")


🚀 Starting training...
Total training samples: 9912027
Batch size: 16
Gradient accumulation steps: 2
Effective batch size: 32
Epochs: 2
Steps per epoch: 309750
Total steps: 619500


In [22]:
# Disable wandb if it's trying to initialize
os.environ['WANDB_DISABLED'] = 'true'

print("\n⏳ Training started - this may take a while...")
try:
    trainer.train()
    print("✅ Training completed successfully!")
except Exception as e:
    print(f"❌ Training failed: {str(e)}")
    import traceback
    traceback.print_exc()
    raise


⏳ Training started - this may take a while...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
20,0.197700
40,0.096200
60,0.064400
80,0.061300
100,0.056600
120,0.053200
140,0.050700
160,0.048600
180,0.051200
200,0.046300


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# 9. Save the model
print("\n💾 Saving fine-tuned model...")
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✅ Model saved to {OUTPUT_DIR}")

In [ ]:
# 10. Inference section
print("\n🔍 Testing inference on example sentences...")
print("=" * 60)

In [ ]:
# Load the fine-tuned model for inference
print("Loading fine-tuned model for inference...")
inference_model = AutoModelForSeq2SeqLM.from_pretrained(OUTPUT_DIR)
inference_tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
inference_model.eval()

# Test examples
examples = [
    "La iglesia contiene numerosas obras de harte",
    "En esta parroquia tamvién desarrolla alguna de sus actividades",
    "Otras obras notables shon las pinturas del Cavalier d Arpino",
    "Recivió la consideración de parroquia en ES_UN_NUMERO"
]

In [ ]:
print("\n🧪 Running inference on test examples:")
print("-" * 50)

In [ ]:
def correct_spelling(text):
    inputs = inference_tokenizer(
        text, 
        return_tensors="pt", 
        max_length=MAX_LENGTH, 
        truncation=True, 
        padding=True
    )
    
    with torch.no_grad():
        outputs = inference_model.generate(
            **inputs,
            max_length=MAX_LENGTH,
            num_beams=4,
            early_stopping=True,
            do_sample=False
        )
    
    corrected = inference_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected

for i, example in enumerate(examples, 1):
    print(f"\n{i}. Input:  {example}")
    corrected = correct_spelling(example)
    print(f"   Output: {corrected}")

print("\n" + "=" * 60)
print("✅ Fine-tuning and inference completed successfully!")
print(f"📁 Model saved in: {OUTPUT_DIR}")

In [ ]:
# Optional: Quick evaluation on test set
print("\n📊 Optional: Quick evaluation on test set...")
test_files = ["test_medicine.tsv", "test_wikicorpus.tsv"]

for test_file in test_files:
    test_path = os.path.join(DATA_DIR, test_file)
    if os.path.exists(test_path):
        print(f"\nLoading {test_file}...")
        test_df = pd.read_csv(test_path, delimiter='\t').dropna()
        
        if len(test_df) > 0:
            # Test on first 5 samples
            print(f"Testing on first 5 samples from {test_file}:")
            for idx in range(min(5, len(test_df))):
                source = test_df.iloc[idx]['source']
                target = test_df.iloc[idx]['target']
                predicted = correct_spelling(source)
                
                print(f"\n  Sample {idx+1}:")
                print(f"    Source:    {source}")
                print(f"    Target:    {target}")
                print(f"    Predicted: {predicted}")
                print(f"    Match:     {'✅' if predicted.strip() == target.strip() else '❌'}")
    else:
        print(f"❌ {test_file} not found for evaluation")

print("\n🎉 Script execution completed!")